In [ ]:
!pip install cvxpy pandas numpy


In [ ]:
import cvxpy as cp
import numpy as np
import pandas as pd



In [5]:
# Change the URL to download the CSV directly
url = "https://docs.google.com/spreadsheets/d/1y0mBNDZOQFXs84qa0u5Ws86WhT46qzkVS1y059f13C8/export?format=csv"

# Read the CSV with specified options
df = pd.read_csv(
    url,
    index_col=None,            # Set the first column as rownames
)

df.head(5)


,Industry,Company,Yeild,Expense Ratio (%),Debt Equity Ratio,EPS,P/E Ratio,PEG Ratio,PB Ratio,Price/Cash Flow Ratio,Price/Sales Ratio,Price/Cashflow,Beta,Alpha,CAGR (1 yr),CAGR (3 yr),CAGR (5 yr),Additional Comments
0,Healthcare,UnitedHealth Group Incorporated (UNH),1.48,NaN,NaN,15.12,36.89,1.54,5.25,NaN,1.36,NaN,0.59,NaN,6.66,27.27,NaN,1
1,Energy,Johnson & Johnson (JNJ),3.1,NaN,NaN,5.98,26.46,0.95,5.50,NaN,4.44,NaN,0.52,NaN,7.87,4.43,NaN,2
2,Finance,AbbVie Inc. (ABBV),3.22,NaN,NaN,2.83,69.85,0.47,53.05,NaN,6.50,NaN,0.62,NaN,46.94,91.05,NaN,2
3,Healthcare,"Merck & Co., Inc. (MRK)",3.02,NaN,NaN,4.78,21.31,0.07,5.93,NaN,4.10,NaN,0.40,NaN,1.15,25.39,NaN,3
4,Energy,Thermo Fisher Scientific Inc. (TMO),0.28,NaN,NaN,15.89,34.98,2.10,4.36,NaN,6.06,NaN,0.78,NaN,22.48,-12.90,NaN,4


In [ ]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19'],
      dtype='object')

In [9]:
# Convert grades to weights for fundamental factors, returns, and beta
grades = df["Additional Comments"].values
betas = df["Beta"].values
returns = df["CAGR (1  yr)"].values
n_stocks = len(df)

# Define optimization variables
weights = cp.Variable(n_stocks)

# Risk is defined by the portfolio beta
portfolio_beta = cp.sum(cp.multiply(betas, weights))

# Expected portfolio return
portfolio_return = cp.sum(cp.multiply(returns, weights))

portfolio_grades=cp.sum(cp.multiply(grades, weights))

# Objective function: Minimize beta (risk) while maximizing returns
# We'll use a weighted objective function that can be adjusted as needed
objective = cp.Maximize(0.3*portfolio_return +0.4*portfolio_grades+ 0.2 * portfolio_beta)

# Constraints:
constraints = [
    cp.sum(weights) == 1,  # Total weights should sum to 1
    weights >= 0,          # No shorting (long-only portfolio)
]

# Add industry diversity constraint, e.g., limit weights per industry if needed
for industry in df["Industry"].unique():
    # Create a boolean mask for the industry
    industry_mask = (df["Industry"] == industry).values
    # Apply the mask to weights
    constraints.append(cp.sum(weights[industry_mask]) <= 0.4)  # Max 20% per industry

for companys in df["Company"].unique():
    # Create a boolean mask for the industry
    company_mask = (df["Company"] == industry).values
    # Apply the mask to weights
    constraints.append(cp.sum(weights[company_mask]) <= 0.04)  # Max 4% per company


In [10]:
#"Industry",


# Formulate the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display results
print("Optimal weights for each stock:")
df["weight"] = weights.value
#print(df[["Industry", "weight"]])
print(df[["Industry","Company", "Beta", "CAGR (1  yr)","weight"]])

Optimal weights for each stock:
      Industry                                Company  Beta  CAGR (1  yr)  \
0   Healthcare  UnitedHealth Group Incorporated (UNH)  0.59          6.66   
1       Energy                Johnson & Johnson (JNJ)  0.52          7.87   
2      Finance                     AbbVie Inc. (ABBV)  0.62         46.94   
3   Healthcare                Merck & Co., Inc. (MRK)  0.40          1.15   
4       Energy    Thermo Fisher Scientific Inc. (TMO)  0.78         22.48   
5      Finance            Eli Lilly and Company (LLY)  0.42         42.93   
6   Healthcare                      Amgen Inc. (AMGN)  0.60         20.97   
7       Energy              Abbott Laboratories (ABT)  0.72         25.21   
8      Finance        Intuitive Surgical, Inc. (ISRG)  1.39         82.18   
9   Healthcare                      Pfizer Inc. (PFE)  0.62         -5.96   
10      Energy              Danaher Corporation (DHR)  0.86         25.44   

          weight  
0  -3.072336e-11  
1  -3